In [69]:
import pymongo
import pandas as pd
import datetime
import math
from dateutil.parser import parse as parseDate
import sys
from fuzzysearch import find_near_matches
sys.path.append('/home/immichail/anaconda3/lib/python3.8/site-packages')

db = pymongo.MongoClient()['ltcLongevity']

In [6]:
users = pd.read_excel('./data/2ndIter/Пользователи.xlsx')

In [8]:
users.columns

Index(['уникальный номер', 'дата создание личного дела', 'Фамилия', 'Имя',
       'Отчетсво', 'пол', 'дата рождения', 'адрес проживания'],
      dtype='object')

In [11]:
users = users.rename(columns = {
    'уникальный номер': 'userId', 
    'дата создание личного дела': 'dateCreated', 
    'Фамилия': 'surName', 
    'Имя': 'name', 
    'Отчетсво': 'thirdName', 
    'пол': 'gender', 
    'дата рождения': 'dateBirth', 
    'адрес проживания': 'address'
})

In [14]:
db.usersV2.insert_many(users.to_dict(orient = 'records'))

In [107]:
df = pd.read_excel('./data/Расписание занятий(1).xlsx')

In [108]:
dfData = df.to_dict(orient = 'records')

df.columns

Index(['уникальный номер', 'направление 1', 'направление 2', 'направление 3',
       'адрес площадки', 'Округ', 'район', 'расписание в активных периодах 1',
       'расписание в закрытых периодах 1', 'расписание в плановом периоде',
       'Дата начала', 'Дата окончания', 'день 1', 'Время начала 1',
       'Время окончания 1', 'Время перерыва в минутах 1', 'день 2',
       'Время начала 2', 'Время окончания 2', 'Время перерыва в минутах 2',
       'день 3', 'Время начала 3', 'Время окончания 3',
       'Время перерыва в минутах 3', 'день 4', 'Время начала 4',
       'Время окончания 4', 'Время перерыва в минутах 4', 'день 5',
       'Время начала 5', 'Время окончания 5', 'Время перерыва в минутах 5',
       'день 6', 'Время начала 6', 'Время окончания 6',
       'Время перерыва в минутах 6', 'день 7', 'Время начала 7',
       'Время окончания 7', 'Время перерыва в минутах 7'],
      dtype='object')

In [109]:
df

,уникальный номер,направление 1,направление 2,направление 3,адрес площадки,Округ,район,расписание в активных периодах 1,расписание в закрытых периодах 1,расписание в плановом периоде,...,Время окончания 5,Время перерыва в минутах 5,день 6,Время начала 6,Время окончания 6,Время перерыва в минутах 6,день 7,Время начала 7,Время окончания 7,Время перерыва в минутах 7
0,801357270,Физическая активность,ОФП,ОФП,"город Москва, Саратовская улица, дом 16, корпус 2",Юго-Восточный административный округ,муниципальный округ Текстильщики,NaN,"c 01.01.2023 по 31.03.2023, Пн., Ср. 19:10-20:...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,801356857,Физическая активность,ОФП,ОФП,"город Москва, Подольская улица, дом 5",Юго-Восточный административный округ,муниципальный округ Марьино,NaN,"c 09.01.2023 по 31.03.2023, Вт., Чт. 10:00-11:...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,801351684,Физическая активность,ОФП,ОФП,"г. Москва, Базовская улица, дом 15, строение 13",Северный административный округ,муниципальный округ Западное Дегунино,NaN,"c 09.01.2023 по 31.03.2023, Вт., Чт. 19:00-20:...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,801353683,Физическая активность,ОФП,ОФП,"город Москва, улица Обручева, дом 28А",Юго-Западный административный округ,муниципальный округ Обручевский,NaN,"c 09.01.2023 по 31.03.2023, Пн., Ср. 13:30-14:...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,801352164,Физическая активность,ОФП,ОФП,"город Москва, Воронцовский парк, дом 3",Юго-Западный административный округ,муниципальный округ Обручевский,NaN,"c 10.01.2023 по 28.02.2023, Вт., Пт. 12:00-13:...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30482,801356595,Образование,Пеший лекторий,Краеведение и пешие прогулки,Центральный административный округ,муниципальный округ Тверской,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30483,801351382,Образование,Пеший лекторий,Краеведение и пешие прогулки,Южный административный округ,муниципальный округ Орехово-Борисово Северное,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30484,801356595,Образование,Пеший лекторий,Краеведение и пешие прогулки,Центральный административный округ,муниципальный округ Таганский,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30485,801351382,Образование,Пеший лекторий,Краеведение и пешие прогулки,Западный административный округ,муниципальный округ Дорогомилово,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
item = dfData[0]

In [85]:
dfDataP = []

for item in dfData:
    itemP = {
        'groupId': item['уникальный номер'],
        'd1LevelName': item['направление 1'],
        'd2LevelName': item['направление 2'],
        'd3LevelName': item['направление 3'],
        'address': item['адрес площадки'],
        'district': item['Округ'],
        'area': item['район'],
        'scheduleActive': item['расписание в активных периодах 1'],
        'scheduleClosed': item['расписание в закрытых периодах 1'],
        'schedulePlanned': item['расписание в плановом периоде'],
        'dateStarted': item['Дата начала'].to_pydatetime(),
        'dateFinished': parseDate(item['Дата окончания'].replace('.012.', '.12.'))
    }
    
    dates = []

    notNaTimeStart = None
    notNaTimeEnd = None

    for i in range(1, 8):
        if isinstance(item['день %d'%i], str):
            dates.append({
                'day': item['день %d'%i],
                'timeStart': item['Время начала %d'%i],
                'timeEnd': item['Время окончания %d'%i],
                'timeBreak': item['Время перерыва в минутах %d'%i]
            })

            if (isinstance(item['Время начала %d'%i], datetime.time)):
                notNaTimeStart = item['Время начала %d'%i].isoformat()
            if (isinstance(item['Время окончания %d'%i], datetime.time)):
                notNaTimeEnd = item['Время окончания %d'%i].isoformat()

    for d in dates:
        if not(isinstance(d['timeStart'], str)):
            d['timeStart'] = notNaTimeStart
        if not(isinstance(d['timeEnd'], str)):
            d['timeEnd'] = notNaTimeStart

    itemP['schedule'] = dates
    
    dfDataP.append(itemP)

In [87]:
db.schedule.insert_many(dfDataP)

In [55]:
[
    {
        'typeGroup': 'Для ума',
        'activities': ['activityId1', 'activityId2', ...]
    },
    {
        'typeGroup': 'Для души',
        'activities': ['activityId1', 'activityId2', ...]
    },
    {
        'typeGroup': 'Для тела',
        'activities': ['activityId1', 'activityId2', ...]
    },
    {
        'typeGroup': 'Для ума',
        'activities': ['activityId1', 'activityId2', ...]
    }
]

{'typeGroup': 'Для ума',
 'activities': ['activityId1', 'activityId2', Ellipsis]}

In [90]:
df = pd.read_excel('./data/Справочник активностей(1).xlsx')

In [97]:
dfData = df.to_dict(orient = 'records')

df.columns

Index(['Разметка: Для ума/ Для души / Для тела', 'id_level1', 'level1',
       'id_level2', 'level2', 'id_level3', 'leve3', 'Признак "Онлайн"',
       'Признак "Выдается диплом"', 'd_level1', 'd_level2', 'd_level3'],
      dtype='object')

In [101]:
dfDataP = []

for item in dfData:
    itemP = {
        'd0LevelName': item['Разметка: Для ума/ Для души / Для тела'],
        'd1LevelId': item['id_level1'],
        'd1LevelName': item['level1'],
        'd2LevelId': item['id_level2'],
        'd2LevelName': item['level2'],
        'd3LevelId': item['id_level3'],
        'd3LevelName': item['leve3'],
        'online': item['Признак "Онлайн"'] == 'Да',
        'certificate': item['Признак "Выдается диплом"'] == 'Да',
        'title': item['d_level1']
    }
    
    dfDataP.append(itemP)

In [120]:
Counter([i['d3LevelName'] for i in db.activities.find({}, {'d3LevelName': 1})])

Counter({'ОНЛАЙН Английский язык': 3,
         'Викторины': 1,
         'Квест': 1,
         'Брейн-ринг': 1,
         'Современные настольные игры': 1,
         'Иные настольные игры': 1,
         'Русское лото': 1,
         'ОНЛАЙН Викторины': 1,
         'ОНЛАЙН Иные интеллектуальные игры': 1,
         'ОНЛАЙН Квест': 1,
         'ОНЛАЙН Брейн-ринг': 1,
         'ОНЛАЙН Иные настольные игры': 1,
         'ОНЛАЙН Русское лото': 1,
         'ОНЛАЙН Современные настольные игры': 1,
         'ОНЛАЙН Шашки': 1,
         'ОНЛАЙН Шахматы': 1,
         'ОНЛАЙН Шахматы и шашки': 1,
         'Шахматы и шашки': 1,
         'Шашки': 1,
         'Шахматы': 1,
         'Английский язык разговорный': 1,
         'Английский язык': 2,
         'Английский язык для начинающих': 1,
         'Правильное питание': 1,
         'Здорово жить': 1,
         'Иппотерапия': 1,
         'Здоровый образ жизни (ЗОЖ)': 1,
         'Здоровый сон': 1,
         'Здоровое похудение': 1,
         'Арабский язык': 1,


In [123]:
def fuzzySearch(q, d3LevelNames: list):
    d3LevelNamesSearch = []

    for n in d3LevelNames:
        try:
            minDist = sum([
                sorted(find_near_matches(qq.lower(), n.lower(), max_l_dist = len(qq) - 1), key = lambda x: x.dist)[0].dist
                for qq in q.split()
            ])

            d3LevelNamesSearch.append({
                'name': n, 
                'minDist': minDist
            })
        except:
            pass
        
    return d3LevelNamesSearch

'ОНЛАЙН Английский язык'

In [249]:

def searchFilters(filters, limit = 10, offset = 0):

    filtersActivities = {}

    if ('d0LevelName' in filters)and(filters['d0LevelName'] is not None):
        filtersActivities['d0LevelName'] = {'$in': filters['d0LevelName']}

    if ('d1LevelName' in filters)and(filters['d1LevelName'] is not None):
        filtersActivities['d1LevelName'] = {'$in': filters['d1evelName']}

    if ('d2LevelName' in filters)and(filters['d2LevelName'] is not None):
        filtersActivities['d2LevelName'] = {'$in': filters['d2LevelName']}

    if ('online' in filters)and(filters['online'] is not None):
        filtersActivities['online'] = filters['online']

    if ('certificate' in filters)and(filters['certificate'] is not None):
        filtersActivities['certificate'] = filters['certificate']

    if len(filtersActivities) > 0:
        d3LevelNames = set([i['d3LevelName'] for i in db.activities.find(filtersActivities, {'d3LevelName': 1})])
    else:
        d3LevelNames = None

    if ('q' in filters)and(filters['q'] is not None):
        if d3LevelNames is None:
            d3LevelNames = set([i['d3LevelName'] for i in db.activities.find({}, {'d3LevelName': 1})])

        d3LevelNamesRelevance = fuzzySearch(filters['q'], d3LevelNames)

        d3LevelNamesRelevance = {i['name']: i['minDist'] for i in d3LevelNamesRelevance}
    else:
        d3LevelNamesRelevance = None

    filtersSchedule = {}

    if ('days' in filters)and(filters['days'] is not None):
        filtersSchedule['schedule.day'] = {'$in': filters['days']}

    if ('district' in filters)and(filters['district'] is not None):
        filtersSchedule['district'] = {'$in': filters['district']}

    if ('area' in filters)and(filters['area'] is not None):
        filtersSchedule['area'] = {'$in': filters['area']}

    if d3LevelNames is not None:
        filtersSchedule['d3LevelName'] = {'$in': list(d3LevelNames)}

    res = list(db.schedule.find(filtersSchedule))

    if d3LevelNamesRelevance is not None:
        res = [r for r in res if r['d3LevelName'] in d3LevelNamesRelevance]
        res = sorted(res, key = lambda x: d3LevelNamesRelevance[x['d3LevelName']])

    res = res[offset:offset + limit]  

    return res

# if filters['d0LevelName'] is not None:
#     d3LevelNameList = set([i['d3LevelName'] for i in db.activities.find(
#         {'d0LevelName': {'$in': filters['d0LevelName']}}
#     )])
    
# if filters['d1LevelName'] is not None:
#     d3LevelNameList_ = set([i['d3LevelName'] for i in db.activities.find(
#         {'d1LevelName': {'$in': filters['d1LevelName']}}
#     )])
    
#     if d3LevelNameList is not None:
#         d3LevelNameList = [d for d in d3LevelNameList if d in d3LevelNameList_]
        
# if filters['d2LevelName'] is not None:
#     d3LevelNameList_ = set([i['d3LevelName'] for i in db.activities.find(
#         {'d2LevelName': {'$in': filters['d2LevelName']}}
#     )])
    
#     if d3LevelNameList is not None:
#         d3LevelNameList = [d for d in d3LevelNameList if d in d3LevelNameList_]
        
# if filters['online'] is not None:
    
#     d3LevelNameList_ = set([i['d3LevelName'] for i in db.activities.find(
#         {'online': filters['online']}
#     )])
    
#     if d3LevelNameList is not None:
#         d3LevelNameList = [d for d in d3LevelNameList if d in d3LevelNameList_]

# if filters['certificate'] is not None:
    
#     d3LevelNameList_ = set([i['d3LevelName'] for i in db.activities.find(
#         {'certificate': filters['certificate']}
#     )])
    
#     if d3LevelNameList is not None:
#         d3LevelNameList = [d for d in d3LevelNameList if d in d3LevelNameList_]

In [252]:
d0LevelName = ['Для ума']

filters = {
    'q': None,
    'd0LevelName': ['Для ума'],
    'd1LevelName': None,
    'd2LevelName': None,
    'online': None,
    'certificate': None,
    'days': None,
    'dates': None,
    'district': None,
    'area': None
}

limit = 10
offset = 0

searchFilters({
    'q': 'англ',
    'd0LevelName': ['Для ума'],
    'online': True,
    'days': ['Сб']
})

[{'_id': ObjectId('6470adcb0ddba6d6eda37093'),
  'groupId': 801354377,
  'd1LevelName': 'Образование',
  'd2LevelName': 'ОНЛАЙН Английский язык',
  'd3LevelName': 'ОНЛАЙН Английский язык',
  'address': 'г. Москва, Новокосинская улица, дом 13, корпус 1',
  'district': 'Восточный административный округ',
  'area': 'муниципальный округ Новокосино',
  'scheduleActive': nan,
  'scheduleClosed': 'c 16.01.2023 по 31.03.2023, Сб. 10:00-12:00, без перерыва',
  'schedulePlanned': nan,
  'dateStarted': datetime.datetime(2023, 1, 1, 0, 0),
  'dateFinished': datetime.datetime(2023, 12, 31, 0, 0),
  'schedule': [{'day': 'Сб',
    'timeStart': '10:00:00',
    'timeEnd': '10:00:00',
    'timeBreak': 0}]},
 {'_id': ObjectId('6470adcb0ddba6d6eda371b1'),
  'groupId': 801354396,
  'd1LevelName': 'Образование',
  'd2LevelName': 'ОНЛАЙН Английский язык',
  'd3LevelName': 'ОНЛАЙН Английский язык',
  'address': 'город Москва, Открытое шоссе, дом 24, корпус 27',
  'district': 'Восточный административный округ'

In [262]:
def getCategories(param: str):
    if param in ['d0LevelName', 'd1LevelName', 'd2LevelName']:
        return db.activities.distinct(param)
    if param in ['area', 'district']:
        return sorted([i.strip() for i in db.schedule.distinct(param) if (isinstance(i, str))and(i.strip() != '')])
    if param in ['online', 'certificate']:
        return ['Да'|"Нет"]
    if param in days:
        return ['Пн', 'Вт', 'Ср', 'Чт', 'Пт', 'Сб', 'Вс']
    
getCategories('d2LevelName')

['Английский язык',
 'Борьба',
 'Велоспорт',
 'ГТО',
 'Гимнастика',
 'Домоводство',
 'Здорово жить',
 'Иностранные языки',
 'Интеллектуальные игры',
 'Интеллектуальный клуб. Домоводство',
 'Интеллектуальный клуб. Здоровье/красота/стиль',
 'Интеллектуальный клуб. Иностранные языки',
 'Интеллектуальный клуб. Информационные технологии',
 'Интеллектуальный клуб. История/Искусство',
 'Интеллектуальный клуб. Клубная работа',
 'Интеллектуальный клуб. Образовательный практикум',
 'Интеллектуальный клуб. Проф/подготовка',
 'Интеллектуальный клуб. Психологические тренинги',
 'Интеллектуальный клуб. Творческие мастерские',
 'Информационные технологии',
 'История, искусство, краеведение',
 'Киберспорт',
 'Коньки',
 'Красота и стиль',
 'Литература',
 'Лыжи',
 'Московский театрал',
 'Музыка, фольклор',
 'Настольные игры',
 'ОНЛАЙН Английский язык',
 'ОНЛАЙН Гимнастика',
 'ОНЛАЙН Домоводство',
 'ОНЛАЙН Здорово жить',
 'ОНЛАЙН Иностранные языки',
 'ОНЛАЙН Интеллектуальные игры',
 'ОНЛАЙН Интеллектуаль